In [8]:
from pyomo.environ import *
import itertools

In [9]:
# Initialize the model
model = ConcreteModel()
# Sets
plants = [1, 2, 3]  # Factories
centers = list(range(1, 13))  # Distribution centers
scenarios = list(itertools.product(['L', 'H'], repeat=len(centers)))  # 2^12 scenarios
# Data
capacity = {1: 120, 2: 90, 3: 100}  # Plant capacities
prod_cost = {1: 8, 2: 7, 3: 9}  # Production cost per unit
sell_price = 20
scrap_price = 5
lost_sale_penalty = 7
# Medium demand
medium_demand = {1: 20, 2: 30, 3: 40, 4: 35, 5: 15, 6: 19, 7: 25, 8: 20, 9: 18, 10: 12, 11: 30, 12: 40}

In [10]:
# Generate demand scenarios
scenario_prob = {}  # Probability of each scenario
scenario_demand = {}  # Demand in each scenario
total_capacity = sum(capacity.values())  # Total production capacity across plants
high_demand_scenarios = 0  # Counter for scenarios where demand > capacity
for scenario in scenarios:
    prob = 1
    demand = {}
    total_demand = 0
    for i, state in enumerate(scenario):
        center = i + 1
        if state == 'L':
            demand[center] = 0.7 * medium_demand[center]
            prob *= 0.5  # Probability of low demand
        else :
            demand[center] = 1.3 * medium_demand[center]
            prob *= 0.5  # Probability of high demand
#         else:
#             demand[center] = medium_demand[center]
#             prob *= 0.4  # Probability of medium demand
        total_demand += demand[center]  # Sum up total demand in scenario

    if total_demand > total_capacity:
        high_demand_scenarios += 1  # Count scenarios exceeding capacity

    scenario_prob[scenario] = prob
    scenario_demand[scenario] = demand
# Calculate fraction of high-demand scenarios
fraction_high_demand = high_demand_scenarios / len(scenarios)

In [11]:
# Decision Variables
model.x = Var(plants, within=NonNegativeReals)  # First-stage decision (production at plants)
model.y = Var(plants, centers, scenarios, within=NonNegativeReals)  # Second-stage (shipments)
model.S = Var(plants, scenarios, within=NonNegativeReals)  # Unsold bread
model.l = Var(centers, scenarios, within=NonNegativeReals)  # Lost sales

In [12]:
# Objective: Maximize expected profit
def objective_rule(m):
    revenue = sum(
        scenario_prob[s] * sell_price * sum(m.y[p, c, s] for p in plants for c in centers)
        for s in scenarios
    )
    production_cost = sum(m.x[p] * prod_cost[p] for p in plants)
    scrap_revenue = sum(
        scenario_prob[s] * scrap_price * sum(m.S[p, s] for p in plants)
        for s in scenarios
    )
    lost_sales_cost = sum(
        scenario_prob[s] * lost_sale_penalty * sum(m.l[c, s] for c in centers)
        for s in scenarios
    )
    return revenue + scrap_revenue - production_cost - lost_sales_cost
model.obj = Objective(rule=objective_rule, sense=maximize)

In [14]:
# Production capacity constraints
def production_capacity_rule(m, p):
    return m.x[p] <= capacity[p]
model.production_capacity = Constraint(plants, rule=production_capacity_rule)
# Flow balance: What is produced must be used (either shipped or left unsold)
def balance_rule(m, p, s):
    return m.x[p] == sum(m.y[p, c, s] for c in centers) + m.S[p, s]
model.balance = ConstraintList()
for p in plants:
    for s in scenarios:
        model.balance.add(balance_rule(model, p, s))
# Demand satisfaction constraints (shipments + lost sales must equal demand)
def demand_rule(m, c, s):
    return sum(m.y[p, c, s] for p in plants) + m.l[c, s] == scenario_demand[s][c]
model.demand = ConstraintList()
for c in centers:
    for s in scenarios:
        model.demand.add(demand_rule(model, c, s))

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.constraint.ConstraintList'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.ConstraintList'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.ConstraintList'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.ConstraintList'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [16]:
# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)
# Display results
print("\nFraction of scenarios where demand exceeds total capacity:", fraction_high_demand)
print("\nOptimal Production Quantities:")
for p in plants:
    print(f"Plant {p}: {model.x[p]():.2f} units")
print("\nExpected Profit: {:.2f}".format(model.obj()))


Fraction of scenarios where demand exceeds total capacity: 0.412109375

Optimal Production Quantities:
Plant 1: 120.00 units
Plant 2: 90.00 units
Plant 3: 100.00 units

Expected Profit: 3432.32
